In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sys
sys.path.append("../library")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import qat.lang.AQASM as qlm

In [ ]:
%load_ext qat.core.magic
%matplotlib inline

In [ ]:
global_qlmaas = True
try:
    from qlmaas.qpus import LinAlg
except (ImportError, OSError) as e:
    global_qlmaas = False
from qat.qpus import PyLinalg

In [ ]:
def get_qpu(qlmass=False):
    """
    Function for selecting solver. User can chose between:
    * LinAlg: for submitting jobs to a QLM server
    * PyLinalg: for simulating jobs using myqlm lineal algebra.

    Parameters
    ----------

    qlmass : bool
        If True  try to use QLM as a Service connection to CESGA QLM
        If False PyLinalg simulator will be used

    Returns
    ----------
    
    lineal_qpu : solver for quantum jobs
    """
    if qlmass:
        if global_qlmaas:
            print('Using: LinAlg')
            linalg_qpu = LinAlg()
        else:
            raise ImportError("""Problem Using QLMaaS.Please create config file or use mylm solver""")
            
    else:
        print('Using PyLinalg')
        linalg_qpu = PyLinalg()
    return linalg_qpu

In [ ]:
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg
QLMaaS = True
linalg_qpu = get_qpu(QLMaaS)

## 1. Quantum Multiplexors

Implementation of loading data routines using the *Lov Grover and Terry Rudolph* routines directly, using controlled rotations by state, is highly inneficient. In general the use of controlled rotations generate highly deep quantum circuits prone to errors. 
A more efficient approach is the use of Quantum Multiplexors where the only controlled gates are c-Not ones. 

The operation we want to implement is a controlled rotation on an angle $\theta_i$ controlled by a state $|i\rangle$. 

For a 2 qbits state the operator should be:

$$
\hat{U}|q_0q_1\rangle= \{ \begin{array}{ll}
      |0\rangle\hat{R}_y(\theta_0)|q_1\rangle  & if \;|q_0\rangle = |0\rangle \\
      |1\rangle\hat{R}_y(\theta_1)|q_1\rangle  & if \;|q_0\rangle = |1\rangle \\
\end{array}
$$

With a quantum multiplexor this operation can be implemented as:

$$\hat{U}(\theta_0, \theta_1)|q_0q_1\rangle= \left( \mathbf{I} \otimes \hat{R}_{y} (\frac{\theta_0+\theta_1}{2}) \right) \hat{C}_{NOT} \left(\mathbf{I} \otimes \hat{R}_{y} (\frac{\theta_0-\theta_1}{2}) \right) \hat{C}_{NOT}|q_0q_1\rangle$$

The circuit representation for this operator is:

![alt text](./QM_01.png)

As can be seen the quantum multiplexor needs only $C_{NOT}$ gates that are implemented in easier way than controlled rotations


For controlled state rotations of more angles quantum multiplexors can be implemented in a recursively way. For example if we want apply a controlled 4 angle rotation $[\theta_0, \theta_1, \theta_2, \theta_3]$ we can use following circuit:

![alt text](./QM_02.png)


In general a quantum multiplexor with $m$ control qubits can be decomposed as 2 multiplexors with $m − 1$ control qubits and 2 $\hat{C}_{NOT}$ gates:
![alt text](./QM_03.png)

In the **QuantumMultiplexors_Module** there are an implementation for a quantum multiplexor controlled by $m$ qbits and the functions neede to use them in order to load probability and a Integral function

## 2. Loading Data Gates: $\mathcal{P}$ and $\mathcal{R}$

First thing we need are the $\mathcal{P}$ and $\mathcal{R}$ for loading probability distribution and integral function respectively into a Quantum State.

In [ ]:
from AuxiliarFunctions import get_histogram

In [ ]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

### 2.1 Loading Probability gate: $\mathcal{P}$

The $\mathcal{P}$ is implemented by the function **load_p_gate** inside the **QuantumMultiplexors_Module**. The input should be a numpy array with the Probability distribution to load into the quantum state. The output of the function is a **qlm** *AbstractGate* customized for the input numpy array which load it into the quantum state. 

First we need to configurate the $\mathcal{P}$ by providing the discretized probability to load

In [ ]:
from QuantumMultiplexors_Module import load_p_gate

In [ ]:
p_gate = load_p_gate(p_X)
%qatdisplay p_gate --depth 1

We can use the *get_results* of **data_extracting** for executing the gate in a  circuit. This functions receives a quantum object, creates the asociated circuit and job. Finally execute the job and get results!!

In [ ]:
from data_extracting import get_results

In [ ]:
#We need to avoid last qbit
pdf, circuit, q_prog, job = get_results(p_gate, linalg_qpu=linalg_qpu, qubits=list(range(p_gate.arity)))

In [ ]:
%qatdisplay circuit --depth 0

In [ ]:
pdf.head()

In order to check if the operation was done properly we can compare the outpus probability of each state and the probabilities wanted to load:

In [ ]:
np.isclose(pdf['Probability'], p_X).all()

### 2.2 Loading Integral Function gate: $\mathcal{F}$


The $\mathcal{F}$ is implemented by an Abstract qlm gate called **load_f_gate**  inside the **QuantumMultiplexors_Module**.. The input should be a numpy array with the function which integral want to be load into the quantum state. The output of the function is a **qlm** *AbstractGate* customized for the input numpy array which load the integral into the last qbit of the quantum state. 


The length of this array should be: $m=2^n$ where n is an integer. The created gate is a $n+1$ gate where the last qbit codifies the integral of the function.

In [ ]:
from QuantumMultiplexors_Module import load_f_gate

In [ ]:
f_gate = load_f_gate(f_X)

In [ ]:
%qatdisplay f_gate --depth 1

For using this gate we need a equal superposition of states

In [ ]:
q_rout = qlm.QRoutine()
q_bit = q_rout.new_wires(f_gate.arity)
#Loading an uniform distribution
for i in range(len(q_bit)-1):
    q_rout.apply(qlm.H, q_bit[i])
q_rout.apply(f_gate, q_bit)

In [ ]:
%qatdisplay q_rout --depth 0

In [ ]:
pdf, circuit, q_prog, job = get_results(q_rout, linalg_qpu=linalg_qpu, qubits=[f_gate.arity-1])

In [ ]:
%qatdisplay circuit --depth 0

In [ ]:
#Integral of f(x)
MeasurementIntegral = pdf['Probability'][1]*2**(n_qbits)

In [ ]:
#Test
np.isclose(MeasurementIntegral, sum(f_X))

### 2.3 Load Complete Data

Now we are going to use $\mathcal{P}$ and $\mathcal{F}$ implemented gates in order to load $E_{x\sim p}(f)$ in a quantum circuit. 

$$E_{x\sim p}(f) = \sum_{x=0}^{2^n-1} p(x)f(x)dx$$

For this the **load_pf_gate**  inside **QuantumMultiplexors_Module** was created. Providing the 2 operator gates $\mathcal{P}$ and $\mathcal{R}$ the function creates a third *Abstract Gate* with the two operators apply consecutively

In [ ]:
from QuantumMultiplexors_Module import load_pf_gate

In [ ]:
pf_gate = load_pf_gate(p_gate, f_gate)
%qatdisplay pf_gate --depth 1

In [ ]:
pdf, circuit, q_prog, job = get_results(pf_gate, linalg_qpu=linalg_qpu, qubits=[pf_gate.arity-1])

In [ ]:
%qatdisplay circuit --depth 0

As explained before the $E_{x\sim p}(f)$ will be loaded in the $|1\rangle$ state of the last qbit of the circuit

In [ ]:
#Integral of f(x)
MeasurementIntegral = pdf['Probability'][1]
print('MeasurementIntegral: {}'.format(MeasurementIntegral))
print('Integral p(x)f(x): {}'.format(sum(p_X*f_X)))

In [ ]:
np.isclose(MeasurementIntegral, sum(p_X*f_X))

## 3 Sumary and Important notes

In **01_Dataloading_Module_Use** Notebook the *dataloading_module* and *dataloading_module_examples* were explained and used. 
Present Notebook try to mimic the **01_Dataloading_Module_Use** one but using and explained **QuantumMultiplexors_Module** and **QuantumMultiplexors_Module_examples**. 

The functions and gates implemented in *dataloading_module* and in the **QuantumMultiplexors_Module** have the same functionality: **load data, probability functions and integral functions** in a quantum state but the implementation was changed:
* In the *dataloading_module*: several controlled by states rotations were straightoforward implemented in order to create the gates to load the data.
* In the **QuantumMultiplexors_Module**: the loading data gates were implemented in a much more efficient way using **quantum multiplexors** where the controlloed rotations are subsituted by simple (and intelligent) qbit Rotations and $\hat{C}_{NOT}$ gates